In [1]:
# coding: utf-8

# # Dynet segmentation with tf fold


#just a bunch of fun
import numpy as np
import six
import time
from multiprocessing import Process, Queue
import time
import data
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
import tensorflow_fold as td
from conv_lstm_cell import *

# params
EMBEDDING_SIZE = 64
SEP = "|"
BATCH_SIZE = 100
data_dir = "/mnt/permanent/Home/nessie/velkey/data/"

#our alphabet

vocabulary = data.vocabulary(data_dir + 'vocabulary')
vsize=len(vocabulary)
print(vocabulary)

index = lambda char: vocabulary.index(char)
char = lambda i: vocabulary[i]




class data():
    def __init__(self, folder, truncate=100):
        self.data_dir = folder
        self.data = dict()
        self.size = dict()
        self.datasets = ["train", "test", "validation"]
        self.truncate = truncate
        
        for dataset in self.datasets:
            self.data[dataset] = self.sentence_reader(folder+dataset)
            #self.size[dataset] = sum(1 for line in open(folder+dataset))

                        
    def sentence_reader(self, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        while True:
            i=0
            sentence = []
            end_sentence = False
            with open(file) as f:
                for lines in f:
                    line = lines[:-1].split('\t')
                    if line[0] != "":
                        sentence.append(line)
                    else:
                        end_sentence = True
                        i+=1
                    if end_sentence:
                        end_sentence = False
                        sent = " ".join([word[0] for word in sentence])
                        segmented = " ".join([word[1].replace(" ","|") for word in sentence])
                        tags = []
                        last_char = "_"
                        for char in segmented:
                            if char != "|":
                                tags.append(0 if last_char!="|" else 1)
                            last_char = char
                        if len(sent) != 0:
                            sent_onehot = self.onehot(sent)
                            if len(sent_onehot) == len(tags):
                                if len(sent_onehot) >= self.truncate:
                                    sent_onehot=sent_onehot[:self.truncate]
                                    tags = tags[:self.truncate]
                                yield (sent_onehot, tags)
                            sentence = []      
          
            
    def onehot(self, string):
        onehot = np.zeros([len(string),vsize])
        indices = np.arange(len(string)), np.array([int(index(char)) for char in string])
        onehot[indices]=1
        return [onehot[i,:] for i in range(len(onehot))]
            
store = data("/mnt/permanent/Home/nessie/velkey/data/")


# ## helper functions

def params_info():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(variable.name, shape)
        # print(len(shape))
        variable_parametes = 1
        for dim in shape:
            # print(dim)
            variable_parametes *= dim.value
        print("\tparams: ", variable_parametes)
        total_parameters += variable_parametes
    print(total_parameters)
    return total_parameters

def onehot(string):
    onehot = np.zeros([len(string),vsize])
    onehot[np.arange(len(string)), np.array([index(char) for char in string])]=1
    return [onehot[i,:] for i in range(len(onehot))]

def convLSTM_cell(kernel_size, out_features = 64):
    convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=out_features, kernel_shape=[kernel_size])
    return td.ScopedLayer(convlstm)

def multi_convLSTM_cell(kernel_sizes, out_features):
    stacked_convLSTM = tf.contrib.rnn.MultiRNNCell()
    return td.ScopedLayer(stacked_convLSTM)

def FC_cell(units):
    return td.ScopedLayer(tf.contrib.rnn.LSTMCell(num_units=units))

def multi_FC_cell(units_list):
    return td.ScopedLayer(tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(num_units=units) for units in units_list]))
    
def bidirectional_dynamic_CONV(fw_cell, bw_cell, out_features=64):
    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():        
        fw_seq = td.Identity().reads(bidir_conv_lstm.input)
        bw_seq = td.Slice(step=-1).reads(fw_seq)

        forward_dir = (td.RNN(fw_cell) >> td.GetItem(0)).reads(fw_seq)
        back_dir = (td.RNN(bw_cell) >> td.GetItem(0)).reads(bw_seq)
        back_to_leftright = td.Slice(step=-1).reads(back_dir)
        
        output_transform = (td.Function(lambda x: tf.reshape(x, [-1,vsize*out_features])) >>
                            #td.FC(10, activation=tf.nn.tanh) >>
                            td.FC(1, activation=None))
        
        bidir_common = (td.ZipWith(td.Concat() >> 
                                  output_transform >> 
                                  td.Metric('logits'))).reads(forward_dir, back_to_leftright)
                    
        #tag_logits = td.Map(output_transform).reads(bidir_common)

        bidir_conv_lstm.output.reads(bidir_common)
    return bidir_conv_lstm

def bidirectional_dynamic_FC(fw_cell, bw_cell, hidden):
    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():        
        fw_seq = td.Identity().reads(bidir_conv_lstm.input)
        bw_seq = td.Slice(step=-1).reads(fw_seq)

        forward_dir = (td.RNN(fw_cell) >> td.GetItem(0)).reads(fw_seq)
        back_dir = (td.RNN(bw_cell) >> td.GetItem(0)).reads(bw_seq)
        back_to_leftright = td.Slice(step=-1).reads(back_dir)
        
        output_transform = td.FC(1, activation=tf.nn.sigmoid)
        
        bidir_common = (td.ZipWith(td.Concat() >> 
                                  output_transform >> 
                                  td.Metric('logits'))).reads(forward_dir, back_to_leftright)

        bidir_conv_lstm.output.reads(bidir_common)
    return bidir_conv_lstm


CONV_data = td.Map(td.Vector(vsize) >> td.Function(lambda x: tf.reshape(x, [-1,vsize,1])))
CONV_model =  CONV_data >> bidirectional_dynamic_CONV(convLSTM_cell(vsize), convLSTM_cell(vsize)) >> td.Void()
#labels = td.Map(td.Scalar() >> td.Metric("labels")) >> td.Void()

FC_data = td.Map(td.Vector(vsize))#>> td.Function(lambda x: tf.reshape(x, [-1,vsize])))
FC_model = FC_data >> bidirectional_dynamic_FC(multi_FC_cell([100]), multi_FC_cell([100]),100) >>td.Void()






[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'C', 'D', 'E', 'G', 'L', 'N', 'S', 'T', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '²', '³', '¹', 'á', 'é', 'í', 'ó', 'ö', 'ú', 'ü', 'ő', 'ű']


In [2]:
def bidirectional_dynamic_FC(fw_cell, bw_cell, hidden):
    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():        
        fw_seq = td.Identity().reads(bidir_conv_lstm.input[0])
        labels = (td.GetItem(1)>>td.Map(td.Metric("labels"))>>td.Void()).reads(bidir_conv_lstm.input)
        bw_seq = td.Slice(step=-1).reads(fw_seq)

        forward_dir = (td.RNN(fw_cell) >> td.GetItem(0)).reads(fw_seq)
        back_dir = (td.RNN(bw_cell) >> td.GetItem(0)).reads(bw_seq)
        back_to_leftright = td.Slice(step=-1).reads(back_dir)
        
        output_transform = td.FC(1, activation=tf.nn.sigmoid)
        
        bidir_common = (td.ZipWith(td.Concat() >> 
                                  output_transform >> td.Metric('logits'))).reads(forward_dir, back_to_leftright)
        
        bidir_conv_lstm.output.reads(bidir_common)
    return bidir_conv_lstm

d = td.Record((td.Map(td.Vector(vsize)),td.Map(td.Scalar())))
f = d >> bidirectional_dynamic_FC(multi_FC_cell([100]), multi_FC_cell([100]),100) >>td.Void()



In [ ]:
f.eval((onehot("naGon jó ötlet"),[0,0,0,1,0,1,0,0,0,0,0,1,0,0]))

In [3]:
compiler = td.Compiler.create(f)
logits = tf.squeeze(compiler.metric_tensors['logits'])
labels = compiler.metric_tensors['labels']

loss = tf.reduce_mean(tf.abs(tf.subtract(labels,logits)))
l2_loss = tf.reduce_mean(tf.abs(tf.subtract(labels,logits)))
log_loss = (labels) * tf.log(logits) + (1 - labels) * tf.log(1 - logits)
#cross_entropy_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels)
best_loss =  labels * -tf.log(logits) + (1 - labels) * -tf.log(1 - logits)
#TODO data label distribution analysis for determining the better best lost ;)


opt = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = opt.minimize(best_loss)
sess.run(tf.global_variables_initializer())


/home/velkey/Envs/tf/lib/python3.4/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
sess.run(tf.global_variables_initializer())
#feed = compiler.build_feed_dict([(onehot("naGon jó ötlet"),[0,0,0,1,0,1,0,0,0,0,0,1,0,0]) for i in range(1)])C
x = next(store.data["train"])
x = next(store.data["train"])
print(x)
feed = compiler.build_feed_dict([x for _ in range(BATCH_SIZE)])
for i in range(100):
    a,b,c,d= sess.run([logits, compiler.metric_tensors['labels'], loss, train_op], feed) #compiler.build_feed_dict([next(store.data["train"]) for _ in range(BATCH_SIZE)]))
    print("step: ", i)
    print("preds: ", a)
    print("labels: ", b)
    print("loss: ", c, '\n')
    accuracy=a > 0.5
    print("accuracy: ", np.sum(np.equal(accuracy,b))/len(accuracy)*100, "%")



([array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,

step:  0
preds:  [ 0.50000399  0.50103194  0.50098634 ...,  0.49946204  0.49987879
  0.50082099]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.500438 

accuracy:  25.0 %
step:  1
preds:  [ 0.49861339  0.49947199  0.49925724 ...,  0.49779975  0.49835664
  0.49927607]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.498847 

accuracy:  87.0 %
step:  2
preds:  [ 0.49729577  0.49800137  0.49760434 ...,  0.49610829  0.4967958
  0.49771228]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.497282 

accuracy:  88.0 %
step:  3
preds:  [ 0.49595302  0.49652576  0.49592382 ...,  0.49424005  0.49508235
  0.49601397]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.495597 

accuracy:  88.0 %
step:  4
preds:  [ 0.49447802  0.4949244   0.49409997 ...,  0.49201697  0.49304011
  0.49400821]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.493612 

accuracy:  88.0 %
step:  5
preds:  [ 0.49274296  0.49306074  0.49200168 ...,  0.48915038  0.49038011
  0.49141413]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]

step:  48
preds:  [ 0.2253868   0.21831398  0.20692489 ...,  0.25616303  0.26674595
  0.27605605]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.215743 

accuracy:  88.0 %
step:  49
preds:  [ 0.22118156  0.2140484   0.20257612 ...,  0.25427526  0.26514858
  0.27468458]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.213464 

accuracy:  88.0 %
step:  50
preds:  [ 0.21750166  0.21031055  0.19875777 ...,  0.25281364  0.26397344
  0.2737326 ]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.211573 

accuracy:  88.0 %
step:  51
preds:  [ 0.21440174  0.20715438  0.19552162 ...,  0.25183618  0.26327786
  0.27325723]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.210099 

accuracy:  88.0 %
step:  52
preds:  [ 0.21190824  0.20460519  0.19289127 ...,  0.25137693  0.263096    0.27329236]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.209057 

accuracy:  88.0 %
step:  53
preds:  [ 0.21002007  0.20266117  0.19086342 ...,  0.2514475   0.26343971
  0.27385038]
labels:  [ 0.  0.  0. ...,  0.  0

step:  96
preds:  [ 0.17319749  0.16364987  0.14624055 ...,  0.28458077  0.3131099
  0.33621368]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.20999 

accuracy:  88.0 %
step:  97
preds:  [ 0.17244996  0.16285501  0.1452014  ...,  0.28447253  0.31353348
  0.33707377]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.210091 

accuracy:  88.0 %
step:  98
preds:  [ 0.17143485  0.16181527  0.14392352 ...,  0.28420898  0.31379628
  0.3377547 ]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.209958 

accuracy:  88.0 %
step:  99
preds:  [ 0.16919729  0.1596283   0.14156796 ...,  0.28372604  0.31383595
  0.33818537]
labels:  [ 0.  0.  0. ...,  0.  0.  0.]
loss:  0.208968 

accuracy:  88.0 %
